In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_6549: Some(http://gw02.itversity.com:4051)

In [2]:
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler}
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.Pipeline

import util.Random

In [3]:
val dataWithoutHeader = spark.read.option("inferSchema", "true").csv("hdfs:///user/kranthidr/dataSets/KDDCup1999DataSet")

Waiting for a Spark session to start...

dataWithoutHeader = [_c0: int, _c1: string ... 40 more fields]


[_c0: int, _c1: string ... 40 more fields]

In [4]:
dataWithoutHeader.take(3)

0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,normal.
0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,2,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,normal.


In [5]:
dataWithoutHeader.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: d

In [6]:
val data = dataWithoutHeader.toDF(
        "duration", "protocol_type", "service", "flag",
        "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
        "hot", "num_failed_logins", "logged_in", "num_compromised",
        "root_shell", "su_attempted", "num_root", "num_file_creations",
        "num_shells", "num_access_files", "num_outbound_cmds",
        "is_host_login", "is_guest_login", "count", "srv_count",
        "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
        "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
        "dst_host_count", "dst_host_srv_count",
        "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
        "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
        "dst_host_serror_rate", "dst_host_srv_serror_rate",
        "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
        "label").cache()

data = [duration: int, protocol_type: string ... 40 more fields]


[duration: int, protocol_type: string ... 40 more fields]

In [7]:
data.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [8]:
val cat_columns = Array("protocol_type", "service", "flag")
var indexers:Array[StringIndexer] = Array()
for (column <- cat_columns) {
    val index = new StringIndexer().setInputCol(column).setOutputCol(column + "_indexed")
    indexers = indexers :+ index
}
var encoders:Array[OneHotEncoder] = Array()
for (column <- cat_columns) {
    val encoder = new OneHotEncoder().setInputCol(column + "_indexed").setOutputCol(column + "_encoded")
    encoders = encoders :+ encoder
}

cat_columns = Array(protocol_type, service, flag)
indexers = Array(strIdx_ca7e563d7d03, strIdx_0ec04a997e80, strIdx_2e24545cdc74)
encoders = Array(oneHot_c1b88428618b, oneHot_b35c216b8c22, oneHot_80f711d05a05)


[oneHot_c1b88428618b, oneHot_b35c216b8c22, oneHot_80f711d05a05]

In [9]:
indexers

[strIdx_ca7e563d7d03, strIdx_0ec04a997e80, strIdx_2e24545cdc74]

In [10]:
encoders

[oneHot_c1b88428618b, oneHot_b35c216b8c22, oneHot_80f711d05a05]

In [11]:
val assembleCols = Set(data.columns: _*) --
      Seq("label", "protocol_type", "service", "flag")--
      Seq("protocol_type_indexed", "service_indexed", "flag_indexed")
    

assembleCols = Set(num_file_creations, root_shell, duration, dst_host_same_srv_rate, dst_host_srv_count, count, dst_host_srv_rerror_rate, land, urgent, serror_rate, src_bytes, logged_in, num_failed_logins, num_shells, su_attempted, is_guest_login, srv_serror_rate, diff_srv_rate, num_outbound_cmds, num_root, num_access_files, dst_host_count, dst_host_rerror_rate, dst_host_srv_serror_rate, dst_host_serror_rate, srv_count, dst_bytes, same_srv_rate, dst_host_diff_srv_rate, hot, rerror_rate, is_host_login, dst_host_srv_diff_host_rate, dst_host_same_src_port_rate, srv_rerror_rate, wrong_fragment, num_compromised, srv_diff_host_rate)


Set(num_file_creations, root_shell, duration, dst_host_same_srv_rate, dst_host_srv_count, count, dst_host_srv_rerror_rate, land, urgent, serror_rate, src_bytes, logged_in, num_failed_logins, num_shells, su_attempted, is_guest_login, srv_serror_rate, diff_srv_rate, num_outbound_cmds, num_root, num_access_files, dst_host_count, dst_host_rerror_rate, dst_host_srv_serror_rate, dst_host_serror_rate, srv_count, dst_bytes, same_srv_rate, dst_host_diff_srv_rate, hot, rerror_rate, is_host_login, dst_host_srv_diff_host_rate, dst_host_same_src_port_rate, srv_rerror_rate, wrong_fragment, num_compromised, srv_diff_host_rate)

In [12]:
 val assembler = new VectorAssembler().
      setInputCols(assembleCols.toArray).
      setOutputCol("featureVector")
    
    val scalar = new StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").
        setWithStd(true).
        setWithMean(false)
    
     val kmeans = new KMeans().
        setSeed(Random.nextLong()).
        setK(180).
        setMaxIter(40).
        setTol(1.0e-5).
        setPredictionCol("cluster").
        setFeaturesCol("scaledFeatureVector")

assembler = vecAssembler_ec78db96d9d0
scalar = stdScal_cf9fd3c1e9e0
kmeans = kmeans_b0684fbb52b9


kmeans_b0684fbb52b9

In [13]:
indexers ++ encoders ++ Array(assembler, scalar, kmeans)

[strIdx_ca7e563d7d03, strIdx_0ec04a997e80, strIdx_2e24545cdc74, oneHot_c1b88428618b, oneHot_b35c216b8c22, oneHot_80f711d05a05, vecAssembler_ec78db96d9d0, stdScal_cf9fd3c1e9e0, kmeans_b0684fbb52b9]

In [14]:
    val pipeline = new Pipeline().setStages(
      indexers ++ encoders ++ Array(assembler, scalar, kmeans))
    
    val pipelineModel = pipeline.fit(data)
    

pipeline = pipeline_6c95c7595f62
pipelineModel = pipeline_6c95c7595f62


pipeline_6c95c7595f62

In [15]:
val transformed_data = pipelineModel.transform(data)

transformed_data = [duration: int, protocol_type: string ... 49 more fields]


[duration: int, protocol_type: string ... 49 more fields]

In [16]:
val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    
kmeansModel.computeCost(transformed_data)/ data.count()

kmeansModel = kmeans_b0684fbb52b9


0.34505630607300264

In [17]:
transformed_data.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [18]:
val clusterLabel = transformed_data.select("cluster","label").as[(Int, String)]

clusterLabel = [cluster: int, label: string]


[cluster: int, label: string]

In [19]:
def entropy(counts: Iterable[Int]): Double = {
    val values = counts.filter(_ > 0)
    val n = values.map(_.toDouble).sum
    values.map(v => -(v/n)*math.log(v/n)).sum
}

entropy: (counts: Iterable[Int])Double


In [20]:
clusterLabel.count()

4898431

In [21]:
clusterLabel.sample(0.5).count()

2447957

In [22]:
clusterLabel.sample(0.000001).show()

+-------+--------+
|cluster|   label|
+-------+--------+
|     10|  smurf.|
|     10|  smurf.|
|     50|neptune.|
|     50|neptune.|
|     17|neptune.|
+-------+--------+



In [23]:
clusterLabel.show(3)

+-------+-------+
|cluster|  label|
+-------+-------+
|    168|normal.|
|    121|normal.|
|    121|normal.|
+-------+-------+
only showing top 3 rows



In [24]:
clusterLabel.getClass()

class org.apache.spark.sql.Dataset

In [25]:
val test = clusterLabel.
      // Extract collections of labels, per cluster
      groupByKey { case (cluster, _) => cluster }.
      mapGroups { case (_, clusterLabels) =>
        val labels = clusterLabels.map { case (_, label) => label }.toSeq
        // Count labels in collections
        val labelCounts = labels.groupBy(identity).values.map(_.size)
        labels.size * entropy(labelCounts)
      }
//toSeq ??
//identity ??

test = [value: double]


[value: double]

In [26]:
test.show(5)

+-----------------+
|            value|
+-----------------+
|63.68774925443596|
|69.65084707608688|
|354.9852695070798|
|              0.0|
|              0.0|
+-----------------+
only showing top 5 rows



In [27]:
val weightedClusterEntropy = clusterLabel.
      // Extract collections of labels, per cluster
      groupByKey { case (cluster, _) => cluster }.
      mapGroups { case (_, clusterLabels) =>
        val labels = clusterLabels.map { case (_, label) => label }.toSeq
        // Count labels in collections
        val labelCounts = labels.groupBy(identity).values.map(_.size)
        labels.size * entropy(labelCounts)
      }.collect()

weightedClusterEntropy = Array(63.68774925443596, 69.65084707608688, 354.9852695070798, 0.0, 0.0, 0.0, 8.616037337020266, 19.105892719140797, 232.55563149876315, 0.0, 0.0, 0.0, 68.69861031408027, 44.33016446758562, 1.9095425048844383, 10.273483584969675, 92.34200628860641, 76.92683474784087, 113.42844664375863, 4633.335023052716, 0.0, 0.0, 286.55244166242034, 0.0, 0.0, 645.62822096013, 0.0, 17.888274940282262, 608.8886911216919, 0.0, 0.0, 232.8458723839342, 34.55327460504769, 9.802296941911234, 0.0, 0.0, 0.0, 0.0, 0.0, 15.745384049326631, 0.0, 32.81206114993599, 52.54487255380844, 57.879476785531374, 9.46663646595531, 35.21047222593583, 0.0, 1071.048526611695, 0.0, 1335.349766281197, 787.105777113715, 0.0, 87.88369148521541, 0.0, 1293.5995214098957, 0.0, 592.2994396545613...


[63.68774925443596, 69.65084707608688, 354.9852695070798, 0.0, 0.0, 0.0, 8.616037337020266, 19.105892719140797, 232.55563149876315, 0.0, 0.0, 0.0, 68.69861031408027, 44.33016446758562, 1.9095425048844383, 10.273483584969675, 92.34200628860641, 76.92683474784087, 113.42844664375863, 4633.335023052716, 0.0, 0.0, 286.55244166242034, 0.0, 0.0, 645.62822096013, 0.0, 17.888274940282262, 608.8886911216919, 0.0, 0.0, 232.8458723839342, 34.55327460504769, 9.802296941911234, 0.0, 0.0, 0.0, 0.0, 0.0, 15.745384049326631, 0.0, 32.81206114993599, 52.54487255380844, 57.879476785531374, 9.46663646595531, 35.21047222593583, 0.0, 1071.048526611695, 0.0, 1335.349766281197, 787.105777113715, 0.0, 87.88369148521541, 0.0, 1293.5995214098957, 0.0, 592.2994396545613, 0.0, 80.8634633681106, 0.0, 866.9853013702398, 821.4837122025122, 0.0, 82.64898643527505, 0.0, 16.161735557597122, 4.1986036933543245, 81.42493330133429, 27.282081801504898, 0.0, 0.0, 120.50075689683277, 8.955249691343504, 0.0, 0.0, 0.0, 102.1537

In [28]:
weightedClusterEntropy.sum/data.count()    /// for k=180

0.008637329852285882

In [29]:
val countByClusterLabel = transformed_data.select("cluster","label").
groupBy("cluster","label").count().orderBy("cluster","label")

countByClusterLabel = [cluster: int, label: string ... 1 more field]


[cluster: int, label: string ... 1 more field]

In [30]:
countByClusterLabel.show()

+-------+-------------+------+
|cluster|        label| count|
+-------+-------------+------+
|      0|        nmap.|    55|
|      0|      normal.| 92953|
|      0|       satan.|    43|
|      0|       smurf.|  1093|
|      1|      normal.|102479|
|      2|    teardrop.|   259|
|      3|   ftp_write.|     2|
|      3|guess_passwd.|     1|
|      3|    multihop.|     2|
|      3|      normal.|  2128|
|      3| warezclient.|     1|
|      3| warezmaster.|     1|
|      4|     neptune.|364025|
|      4|      normal.|    11|
|      5|      normal.|     2|
|      6|        land.|    21|
|      6|      normal.|     7|
|      7|      normal.|     6|
|      8|      normal.|    17|
|      9|      normal.|  6087|
+-------+-------------+------+
only showing top 20 rows



In [31]:
import org.apache.spark.ml.linalg.{Vector, Vectors}

In [32]:
val KmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]

KmeansModel = kmeans_b0684fbb52b9


kmeans_b0684fbb52b9

In [33]:
val centroids = KmeansModel.clusterCenters

centroids = Array([0.0,0.0,0.004902480388564476,2.275436732377507,2.244516623346322,0.07932482976885182,5.518468004621867E-6,0.0,0.0,6.870406066372829E-4,1.0671307353891673E-4,0.0,0.0,0.0,0.0,0.0,6.680203308143666E-4,0.008049228448055033,0.0,0.0,0.0,3.968011694396108,5.61588950291064E-4,9.171835314385382E-6,1.5578154208843668E-5,0.09519767561987116,1.657662238830651E-4,2.5677378089913665,0.08702189793956057,0.0,1.5092462460102963E-5,0.0,4.4022688695659356E-4,0.050399597409161646,1.506602962540001E-5,0.0,0.0,0.027962252184663385], [0.0,0.0,4.0710348443062037E-4,2.4311347544800244,2.3901180762370076,0.03783673215787629,0.005086951618258676,0.0,0.0,0.0031233527252109133,2.9456479796868314E-4,2.852159245902337,0.0,0.0,0.0,0.0,0.0031284289798914023,3...


[[0.0,0.0,0.004902480388564476,2.275436732377507,2.244516623346322,0.07932482976885182,5.518468004621867E-6,0.0,0.0,6.870406066372829E-4,1.0671307353891673E-4,0.0,0.0,0.0,0.0,0.0,6.680203308143666E-4,0.008049228448055033,0.0,0.0,0.0,3.968011694396108,5.61588950291064E-4,9.171835314385382E-6,1.5578154208843668E-5,0.09519767561987116,1.657662238830651E-4,2.5677378089913665,0.08702189793956057,0.0,1.5092462460102963E-5,0.0,4.4022688695659356E-4,0.050399597409161646,1.506602962540001E-5,0.0,0.0,0.027962252184663385], [0.0,0.0,4.0710348443062037E-4,2.4311347544800244,2.3901180762370076,0.03783673215787629,0.005086951618258676,0.0,0.0,0.0031233527252109133,2.9456479796868314E-4,2.852159245902337,0.0,0.0,0.0,0.0,0.0031284289798914023,3.5155987190202995E-4,0.0,1.9823078097691498E-5,0.0,0.34342586837153405,0.008703757764554907,0.0023492817598901867,0.003952937091377023,0.03634820261238812,0.008384659503512313,2.568703192768809,0.0031842781868392602,0.002704928880040211,0.00286121863003926,0.0,0

In [34]:
val threshold = transformed_data.select("cluster","scaledFeatureVector").as[(Int,Vector)].
map { case(cluster, vec) => Vectors.sqdist(centroids(cluster), vec)}.
orderBy($"value".desc).take(100).last


threshold = 828.9728200429139


828.9728200429139

In [35]:
val originalCols = data.columns
val anomalies = transformed_data.filter{ row => 
val cluster = row.getAs[Int]("cluster")
val vec = row.getAs[Vector]("scaledFeatureVector")
Vectors.sqdist(centroids(cluster), vec) >= threshold
}

originalCols = Array(duration, protocol_type, service, flag, src_bytes, dst_bytes, land, wrong_fragment, urgent, hot, num_failed_logins, logged_in, num_compromised, root_shell, su_attempted, num_root, num_file_creations, num_shells, num_access_files, num_outbound_cmds, is_host_login, is_guest_login, count, srv_count, serror_rate, srv_serror_rate, rerror_rate, srv_rerror_rate, same_srv_rate, diff_srv_rate, srv_diff_host_rate, dst_host_count, dst_host_srv_count, dst_host_same_srv_rate, dst_host_diff_srv_rate, dst_host_same_src_port_rate, dst_host_srv_diff_host_rate, dst_host_serror_rate, dst_host_srv_serror_rate, dst_host_rerror_rate, dst_host_srv_rerror_rate, label)
anomalies = [duration: int, protocol_type: string .....


[duration: int, protocol_type: string ... 49 more fields]

In [36]:
originalCols.head

duration

In [37]:
originalCols.tail

[protocol_type, service, flag, src_bytes, dst_bytes, land, wrong_fragment, urgent, hot, num_failed_logins, logged_in, num_compromised, root_shell, su_attempted, num_root, num_file_creations, num_shells, num_access_files, num_outbound_cmds, is_host_login, is_guest_login, count, srv_count, serror_rate, srv_serror_rate, rerror_rate, srv_rerror_rate, same_srv_rate, diff_srv_rate, srv_diff_host_rate, dst_host_count, dst_host_srv_count, dst_host_same_srv_rate, dst_host_diff_srv_rate, dst_host_same_src_port_rate, dst_host_srv_diff_host_rate, dst_host_serror_rate, dst_host_srv_serror_rate, dst_host_rerror_rate, dst_host_srv_rerror_rate, label]

In [38]:
anomalies.count()

100

In [39]:
anomalies.columns.size

51

In [40]:
anomalies.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [41]:
val final_list = anomalies.select(originalCols.head,originalCols.tail:_*)

final_list = [duration: int, protocol_type: string ... 40 more fields]


[duration: int, protocol_type: string ... 40 more fields]

In [42]:
final_list.columns.size

42

In [43]:
final_list.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [44]:
final_list.take(1)

0,tcp,telnet,S1,167,19827,0,0,0,0,0,1,83,1,2,91,0,0,0,0,0,0,1,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,21,8,0.29,0.14,0.05,0.25,0.05,0.12,0.0,0.0,normal.


In [45]:
anomalies.select(originalCols)

Name: Compile Error
Message: <console>:60: error: overloaded method value select with alternatives:
  [U1](c1: org.apache.spark.sql.TypedColumn[org.apache.spark.sql.Row,U1])org.apache.spark.sql.Dataset[U1] <and>
  (col: String,cols: String*)org.apache.spark.sql.DataFrame <and>
  (cols: org.apache.spark.sql.Column*)org.apache.spark.sql.DataFrame
 cannot be applied to (Array[String])
       anomalies.select(originalCols)
                 ^

StackTrace: 